## CatBoost прогноз по складам

In [1]:
%%time         
import warnings             #ТОП прогноз по складам
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

# Блок 1: Загрузка данных и предварительная обработка
data = pd.read_excel('GO data со складами.xlsx')
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)

# Подготовка данных
data = data.rename(columns={'Дата': 'ds', 'Склад1': 'Store1', 'Склад2': 'Store2', 'Отгрузка': 'y'})

# Подготовка пустого DataFrame для прогнозов
forecast_dfs = []

# Функция для добавления лагов и скользящих средних
def create_lag_features(df, lags, window):
    for lag in lags:
        df[f'lag_{lag}'] = df['y'].shift(lag)
    df[f'rolling_mean_{window}'] = df['y'].rolling(window=window).mean()
    return df

# Блок 2: Создание и обучение модели CatBoost
for (store1, store2) in data[['Store1', 'Store2']].drop_duplicates().itertuples(index=False):
    subset = data[(data['Store1'] == store1) & (data['Store2'] == store2)].copy()

    if len(subset) >= 2:
        subset.set_index('ds', inplace=True)
        subset = subset.asfreq('D')
        subset['y'] = subset['y'].fillna(0)

        # Создание дополнительных признаков времени
        subset['day'] = subset.index.day
        subset['month'] = subset.index.month
        subset['year'] = subset.index.year
        subset['day_of_week'] = subset.index.dayofweek

        # Добавление лагов и скользящих средних
        subset = create_lag_features(subset, lags=[1, 7], window=7)

        # Заполнение пропусков, возникших после создания лагов
        subset.fillna(0, inplace=True)

        X = subset[['day', 'month', 'year', 'day_of_week', 'lag_1', 'lag_7', 'rolling_mean_7']]
        y = subset['y']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

        model = CatBoostRegressor(iterations=1500, learning_rate=0.01, depth=8, l2_leaf_reg=3, verbose=0)
        model.fit(X_train, y_train)

        future_dates = pd.date_range(start=subset.index[-1] + pd.DateOffset(1), periods=135, freq='D')       #Период 55 дней
        future = pd.DataFrame(index=future_dates)

        future['day'] = future.index.day
        future['month'] = future.index.month
        future['year'] = future.index.year
        future['day_of_week'] = future.index.dayofweek

        # Добавление лагов для прогноза
         #Лаги (lags) — это сдвиги временного ряда на несколько периодов назад
        future['lag_1'] = subset['y'].iloc[-1]
        future['lag_7'] = subset['y'].iloc[-7] if len(subset) >= 7 else subset['y'].mean()
        future['rolling_mean_7'] = subset['y'].rolling(window=7).mean().iloc[-1]
        
        future['y_pred'] = model.predict(future[['day', 'month', 'year', 'day_of_week', 'lag_1', 'lag_7', 'rolling_mean_7']])

        future['Store1'] = store1
        future['Store2'] = store2
        forecast_dfs.append(future)

final_forecast = pd.concat(forecast_dfs)
final_forecast['y_pred'] = final_forecast['y_pred'].apply(lambda x: 0 if x < 0 else x)

final_forecast.reset_index(inplace=True)
final_forecast.rename(columns={'index': 'ds', 'y_pred': 'yhat'}, inplace=True)
final_forecast[['ds', 'Store1', 'Store2', 'yhat']].to_excel('прогноз.xlsx', index=False)

CPU times: total: 3min 45s
Wall time: 48.2 s


# Прогноз ГО по магазинам

In [ ]:
import warnings               # Прогноз ГО по магазинам
import pandas as pd 
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

# Блок 1: Загрузка данных и предварительная обработка
data = pd.read_excel('GO data без складов.xlsx')
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)

# Подготовка данных
data = data.rename(columns={'Дата': 'ds', 'Магазин': 'Store', 'Отгрузка': 'y'})

# Подготовка пустого DataFrame для прогнозов
forecast_dfs = []

# Функция для добавления лагов и скользящих средних
def create_lag_features(df, lags, window):
    for lag in lags:
        df[f'lag_{lag}'] = df['y'].shift(lag)
    df[f'rolling_mean_{window}'] = df['y'].rolling(window=window).mean()
    return df

# Блок 2: Создание и обучение модели CatBoost
for store in data['Store'].unique():
    subset = data[data['Store'] == store].copy()

    if len(subset) >= 2:
        subset.set_index('ds', inplace=True)
        subset = subset.asfreq('D')
        subset['y'] = subset['y'].fillna(0)

        # Создание дополнительных признаков времени
        subset['day'] = subset.index.day
        subset['month'] = subset.index.month
        subset['year'] = subset.index.year
        subset['day_of_week'] = subset.index.dayofweek

        # Добавление лагов и скользящих средних
        subset = create_lag_features(subset, lags=[1, 7], window=7)

        # Заполнение пропусков, возникших после создания лагов
        subset.fillna(0, inplace=True)
         #Лаги (lags) — это сдвиги временного ряда на несколько периодов назад
        X = subset[['day', 'month', 'year', 'day_of_week', 'lag_1', 'lag_7', 'rolling_mean_7']]
        y = subset['y']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

        model = CatBoostRegressor(iterations=1500, learning_rate=0.01, depth=8, l2_leaf_reg=3, verbose=0)
        model.fit(X_train, y_train)

        # Прогнозирование на следующие 515 дней
        future_dates = pd.date_range(start=subset.index[-1] + pd.DateOffset(1), periods=515, freq='D')
        future = pd.DataFrame(index=future_dates)

        future['day'] = future.index.day
        future['month'] = future.index.month
        future['year'] = future.index.year
        future['day_of_week'] = future.index.dayofweek

        # Добавление лагов для прогноза
        future['lag_1'] = subset['y'].iloc[-1]
        future['lag_7'] = subset['y'].iloc[-7] if len(subset) >= 7 else subset['y'].mean()
        future['rolling_mean_7'] = subset['y'].rolling(window=7).mean().iloc[-1]

        future['y_pred'] = model.predict(future[['day', 'month', 'year', 'day_of_week', 'lag_1', 'lag_7', 'rolling_mean_7']])

        future['Store'] = store
        forecast_dfs.append(future)

# Объединение прогнозов в один DataFrame
final_forecast = pd.concat(forecast_dfs)

# Обработка отрицательных прогнозов
final_forecast['y_pred'] = final_forecast['y_pred'].apply(lambda x: 0 if x < 0 else x)

# Сохранение прогноза в Excel
final_forecast.reset_index(inplace=True)
final_forecast.rename(columns={'index': 'ds', 'y_pred': 'yhat'}, inplace=True)
final_forecast[['ds', 'Store', 'yhat']].to_excel('прогноз_по_магазинам.xlsx', index=False)

# RNN GRU Нейронка

In [ ]:
#RNN GRU нейронка. по скаладам. Не плоха.нужно тестить и убрать минусовые прогнозы
import warnings
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings("ignore")

# Загрузка данных
data = pd.read_excel('GO data со складами.xlsx')
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)

# Подготовка данных
data = data.rename(columns={'Дата': 'ds', 'Склад1': 'Store1', 'Склад2': 'Store2', 'Отгрузка': 'y'})
forecast_dfs = []

# Функция для создания временных последовательностей
def create_sequences(values, time_steps):
    output = []
    for i in range(len(values) - time_steps):
        output.append(values[i:(i + time_steps)])
    return np.array(output)

# Блок 2: Создание и обучение улучшенной модели GRU
for (store1, store2) in data[['Store1', 'Store2']].drop_duplicates().itertuples(index=False):
    subset = data[(data['Store1'] == store1) & (data['Store2'] == store2)].copy()
    
    if len(subset) >= 20:  # Минимум данных для обучения
        subset.set_index('ds', inplace=True)
        subset = subset.asfreq('D')
        subset['y'] = subset['y'].fillna(0)

        # Нормализация данных
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(subset[['y']])

        # Создание временных последовательностей
        time_steps = 7  # Используем данные за последнюю неделю для предсказания
        X = create_sequences(scaled_data, time_steps)
        y = scaled_data[time_steps:]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

        # Построение улучшенной модели GRU
        model = tf.keras.Sequential([
            tf.keras.layers.GRU(100, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.GRU(100, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(1)
        ])

        model.compile(optimizer='adam', loss='mse')
        model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1, validation_data=(X_test, y_test),
                  callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])

        # Прогнозирование на следующие 55 дней
        predictions = []
        current_sequence = X[-1]

        for _ in range(135):                                                   # Прогнозирование на N дней вперед
            pred = model.predict(current_sequence[np.newaxis, :, :])
            predictions.append(pred[0, 0])
            current_sequence = np.roll(current_sequence, -1, axis=0)
            current_sequence[-1] = pred

        # Проверка на соответствие длины дат и предсказаний
        future_dates = pd.date_range(start=subset.index[-1] + pd.DateOffset(1), periods=135, freq='D')    # Прогнозирование на N дней вперед

        if len(future_dates) != len(predictions):
            raise ValueError(f"Length mismatch: {len(future_dates)} dates vs {len(predictions)} predictions")

        # Инвертирование нормализации и замена отрицательных значений на 0
        predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
        predictions = np.maximum(predictions, 0)  # Заменяем отрицательные значения на 0

        # Создание DataFrame для хранения предсказаний
        future = pd.DataFrame({'ds': future_dates, 'yhat': predictions})
        future['Store1'] = store1
        future['Store2'] = store2
        forecast_dfs.append(future)

# Сохранение результата
final_forecast = pd.concat(forecast_dfs)
final_forecast.to_excel('прогнозOK-DEK.xlsx', index=False)